In [ ]:
%load_ext autoreload
%autoreload 2
from pathlib import Path

import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

from pasna_analysis import DataLoader, Experiment, utils

experiment_name = '20240829'
# Use the list below to exclude embryos from the experiment.
# Add embryo ids to the list (based on csv file name).
to_exclude = [1]

exp_path = Path.cwd().parent.joinpath('data',experiment_name)
exp = Experiment(DataLoader(exp_path), first_peak_threshold=0, to_exclude=to_exclude, dff_strategy='local_minima')

In [ ]:
'''Plots activity signals for a group of embryos.'''
# Change start and end to select different groups
start = 0
end = 6

fig, axes = plt.subplots(end-start, 2, figsize=(12, 4*(end-start)))

for (left, right), emb in zip(axes, exp.embryos[start:end]):
    time = emb.activity[:, 0] / 60
    x_points = utils.time_scale_list(max(time))
    trace = exp.traces[emb.name]
    left.set_title(f'{emb.name} - GCamP')
    left.plot(time, trace.active, color='green')
    left.set_ylabel('ΔF/F')
    left.set_xlabel('time (mins)')
    left.set_xticks(x_points)
    right.set_title(f'{emb.name} - tdTomato')
    right.plot(time, trace.struct, color='firebrick')
    right.set_ylabel('ΔF/F')
    right.set_xlabel('time (mins)')
    right.set_xticks(x_points)

    for peak in trace.peak_times:
        peak_in_mins = peak / 60
        left.axvline(peak_in_mins, color='k', alpha=0.3)

plt.tight_layout()

In [ ]:
'''Plots dff for a group of embryos.'''
# Change start and end to select different groups
start = 0
end = 6

fig, axes = plt.subplots(end-start, figsize=(8, 4*(end-start)))
axes = axes.flatten()
for ax, emb in zip(axes, exp.embryos[start:end]):
    time = emb.activity[:, 0] / 60
    x_points = utils.time_scale_list(max(time))
    trace = exp.traces[emb.name]
    ax.set_title(f'{emb.name} - dff')
    ax.set_ylabel('ΔF/F')
    ax.set_xlabel('time (mins)')
    ax.set_xticks(x_points)
    ax.plot(time, trace.dff)
    for peak in trace.peak_times:
        peak_in_mins = peak / 60
        ax.axvline(peak_in_mins, color='k', alpha=0.3)

plt.tight_layout()

In [ ]:
'''Developmental times at first peak.'''
data = {'dev_fp': [], 'emb': []}

for emb in exp.embryos:
    trace = exp.traces[emb.name]
    time_first_peak = trace.peak_times[0]
    dev_time_first_peak = emb.get_DT_from_time(time_first_peak)
    data['dev_fp'].append(dev_time_first_peak)
    data['emb'].append(emb.name)

df = pd.DataFrame(data)

sns.set_theme(style='darkgrid')
ax = sns.swarmplot(data=data, x='emb', y='dev_fp')
ax.set_title('Developmental time at first peak')
ax.set_ylabel('Developmental time')
ax.set_xlabel('Experiment Group');

In [ ]:
'''Peak amplitudes for each episode.'''
num_of_peaks = 13
data = {'peak_amp': [], 'emb': [], 'peak_idx': []}

for emb in exp.embryos:
    t = exp.traces[emb.name]
    for i, amp in zip(range(num_of_peaks), t.peak_amplitudes):
        data['peak_amp'].append(amp)
        data['emb'].append(emb.name)
        data['peak_idx'].append(i)

amps = pd.DataFrame(data)

sns.set_theme(style='darkgrid')
ax = sns.scatterplot(data=data, x='peak_idx', y='peak_amp', hue='emb')
sns.move_legend(ax, 'upper left', bbox_to_anchor=(1,1))
ax.set_title(f'Peak amplitudes')
ax.set_xlabel('Peak number')
ax.set_ylabel('\u0394F/F');

In [ ]:
'''Plots AUC, grouped by bins.'''
data = {'auc': [], 'bin': [], 'emb': []}

n_bins = 5
first_bin = 2
bin_width = 0.2
for i, emb in enumerate(exp.embryos):
    trace = exp.traces[emb.name]

    dev_time_at_peaks = emb.get_DT_from_time(trace.peak_times)

    bins = [first_bin + j*bin_width for j in range(n_bins)]
    bin_idxs = utils.split_in_bins(dev_time_at_peaks, bins)
    data['auc'].extend(trace.peak_aucs)
    data['bin'].extend(bin_idxs)
    data['emb'].extend([str(i)]*len(trace.peak_aucs))

# add a last bin point to generate the labels
bins.append(first_bin + bin_width*n_bins)
x_labels = [f"{s}~{e}" for (s, e) in zip(bins[:-1], bins[1:])]
ax = sns.pointplot(data=data, x='bin', y='auc', linestyle='None')
ax.set_xticks(ticks=list(range(n_bins)), labels=x_labels)
ax.set_title(f'Binned AUC - Exp {exp.name}')
ax.set_ylabel('AUC [activity*t]');

In [ ]:
'''Plot peak times for all embryos. This should help to isolate the 
early peaks.'''
UPPER_LIMIT = 15000
times = []
for emb in exp.embryos:
    trace = exp.traces[emb.name]
    times.append([t/60 for t in trace.peak_times if t < 15000])

fig, ax = plt.subplots()
for i, time in enumerate(times):
    ax.plot(time, [i]*len(time), marker='.', linestyle='dashed', 
            linewidth=0.5)

ax.vlines(30, 0, len(exp.embryos), color='k', linewidth=0.3)
ax.set_title(f'Peak times for exp {exp.name}')
x_points = utils.time_scale_list(UPPER_LIMIT/60)
ax.set_xticks(x_points)
ax.set_xlabel('time (mins)')
ax.set_ylabel('emb')
ax.set_yticks([])

In [ ]:
'''Compare first interspike interval and average interspike interval'''
import numpy as np

avg_ISIs = []
first_ISIs = []
for emb in exp.embryos:
    trace = exp.traces[emb.name]
    peak_times = trace.peak_times
    avg_ISIs.append(np.average(np.diff(peak_times[1:])))
    first_ISIs.append(peak_times[1] - peak_times[0])

max_val = np.max((np.max(avg_ISIs), np.max(first_ISIs)))
diag = list(range(0, int(max_val), 50))

fig, ax = plt.subplots()
ax.plot(avg_ISIs, first_ISIs, 'k.')
ax.plot(diag, diag, label='y=x')
ax.plot(diag, list(range(0, int(max_val)*2, 50*2)), label='y=2*x')
ax.set_xlabel('average ISI')
ax.set_ylabel('first ISI')
ax.set_xlim(0, max_val)
ax.set_ylim(0, max_val)
ax.set_title(f'Differences between first and avg intervals - exp {exp.name}')
ax.legend();